In [1]:
%matplotlib inline

In [12]:
import os
import torch

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [3]:
%load_ext autoreload
%autoreload 2

# load data

In [10]:
from loaders import prepareReportData, tensorsFromPair, normalizeString

In [8]:
input_lang, output_lang, ds = prepareReportData()

read data...
read 3580 reports
trimmed to 3580 sentence pairs
counting words...
counted words:
	 tags 590
	 report 1953


In [13]:
report_pair = ds.__getitem__(100)[:2]
tensorsFromPair((report_pair[0], normalizeString(report_pair[1])), input_lang, output_lang, device, 15)

(tensor([[2],
         [1]], device='cuda:1'),
 tensor([[  2],
         [  3],
         [  9],
         [ 10],
         [ 11],
         [169],
         [  3],
         [  3],
         [ 41],
         [ 94],
         [ 95],
         [ 25],
         [ 20],
         [ 21],
         [ 22]], device='cuda:1'))

# train

In [14]:
from models import EncoderRNN, AttnDecoderRNN

In [15]:
from train_utils import trainIters

In [16]:
%%time
hidden_size = 256
max_length = 30
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, max_length, dropout_p=0.1).to(device)

plot_losses = trainIters(ds, encoder1, attn_decoder1, 100, max_length, input_lang, output_lang, print_every=10)

0m 1s (- 0m 10s) (10 10%) 7.0445
0m 2s (- 0m 9s) (20 20%) 5.8858
0m 3s (- 0m 8s) (30 30%) 6.0647
0m 4s (- 0m 6s) (40 40%) 5.3112
0m 4s (- 0m 4s) (50 50%) 5.5205
0m 5s (- 0m 3s) (60 60%) 4.6969
0m 6s (- 0m 2s) (70 70%) 5.2753
0m 7s (- 0m 1s) (80 80%) 4.4765
0m 8s (- 0m 0s) (90 90%) 4.9247
0m 9s (- 0m 0s) (100 100%) 4.4294
CPU times: user 10.2 s, sys: 316 ms, total: 10.5 s
Wall time: 10.6 s


# eval

In [21]:
from perf import get_av_bleu
from test_utils import evaluateAll, evaluateRandomly

In [27]:
%%time
pred_output = evaluateAll(ds, encoder1, attn_decoder1, max_length, input_lang, output_lang)

CPU times: user 1min 45s, sys: 1.23 s, total: 1min 46s
Wall time: 1min 46s


In [28]:
print('BLEU: {:.2%}'.format(
    get_av_bleu(pred_output)))

/home/alex/anaconda3/envs/bcic1/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU: 3.24%


# WIP plot learning curve

In [47]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)